# Блокнот для написания обработки данных (transform), полученных с newsApi

## Возможные операции трансформации данных:
- Очистка данных (удаление неиспользуемых признаков, дубликатов, выбросов)
- Переформатирование (форматирование данных с разных источников. Форматы дат, валюты и тп)
- Извлечение признаков (создание новых признаков на основе существующих)
- Агрегация (получение необходимых показателей)
- Объединение (объединение данных с нескольких источников)
- Фильтрация (исключение ненужных категорий из набора данных)

In [ ]:
import re
import pandas as pd
import nltk
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer


## 1. Импорт данных

In [ ]:
data = pd.read_csv("data/data_apple.csv", index_col=0)

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.columns

In [ ]:
data.head(3)

## 2. Предобработка

### 2.1 Работа с пропущенными значениями

In [ ]:
data.dropna(subset=["title", "content"], inplace=True)

In [ ]:
data[data["author"].isnull()].head()

In [ ]:
data.dropna(subset=["author"], inplace=True)

Была мысль заменить пустых author на Unknown, но данные записи, как видно, не хранят полезную информацию, поэтому удалим

### 2.2 Дубликаты

In [ ]:
data.duplicated(subset=["title", "content"]).any()

Имеются дубликаты, необходимо удалить

In [ ]:
data[data.duplicated(subset=["title", "content"])]

In [ ]:
data.drop_duplicates(subset=["title", "content"], inplace=True)

### 2.3 Предобработка текста

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def clean_text(text):
    """
    Функция очистки текста
    Удаляет лишние пробелы и символы переноса строк и табуляции
    Оставляет только буквы и цифры

    Args:
        text (_type_): Исходный текст

    Returns:
        _type_: Очищенный текст
    """

    if not isinstance(text, str):
        return ""
    text = re.sub(r"\s+", " ", text) # удаляем символы табуляции и переносов строк
    text = re.sub(r'\[\+\d+ chars\]', '', text) # удаляем [+123 chars] (есть в каждом content)
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # оставляем только буквы и цифры
    
    text = text.lower()
    words = word_tokenize(text)  # Токенизация
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]  # Лемматизация
    return " ".join(words)

data["title"] = data["title"].apply(clean_text)
data["description"] = data["description"].apply(clean_text)
data["content"] = data["content"].apply(clean_text)

### 2.4 Преобразование дат

In [ ]:
data['publishedAt'] = pd.to_datetime(data['publishedAt'], errors='coerce')

In [ ]:
data['date'] = data['publishedAt'].dt.date
data['time'] = data['publishedAt'].dt.time

### 2.5 Фильтрация

In [ ]:
data.drop(columns=["urlToImage", "source.id", "source.name"], inplace=True)
data.head(3)

## 3. Какие задачи необходимо решить:
- Подсчет самых частых слов
- Определение тональности
- Определение главных тем

### 3.1 Подсчет самых частых слов

In [ ]:
from collections import Counter

def find_most_common_words(words, n: int=10):
    word_count = Counter(words)
    sorted_words = word_count.most_common(n)
    return sorted_words

data.head()

all_words = [word for sent in data["content"] for word in sent.split()]

most_common = find_most_common_words(all_words, 10)
most_common

In [ ]:
words, count = zip(*most_common)

In [ ]:
plt.bar(words, count)
plt.title('Топ-10 самых частых слов')
plt.xlabel('Слово')
plt.ylabel('Частота')
plt.xticks(rotation=45)
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

### 3.2 Определение тональности текста

In [ ]:

nltk.download('vader_lexicon')


def get_text_sentiment(text: str) -> str:
    """Функция определения тональности текста

    Args:
        text (str): исходный текст

    Returns:
        str: тональность (neg, neu, pos, compound)
    """
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(text)
    return max(sentiment_scores, key=sentiment_scores.get)

data["sentiment"] = data["description"].apply(get_text_sentiment)
data.head(3)

Ниже решил вспомнить различные варианты визуализации данных

In [ ]:
sentiment_counts = data["sentiment"].value_counts()

plt.bar(sentiment_counts.index, sentiment_counts.values, color=["blue", "gray", "red", "green"])
plt.title('Распределение тональности текстов')
plt.xlabel('Тональность')
plt.ylabel('Частота')
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()


In [ ]:
plt.hist(data["sentiment"], color="blue")
plt.title('Распределение тональности текстов')
plt.xlabel('Тональность')
plt.ylabel('Частота')
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

In [ ]:
plt.pie(sentiment_counts, labels=sentiment_counts.index, colors=["blue", "gray", "red", "green"])
plt.title('Распределение тональности текстов')
plt.xlabel('Тональность')
plt.ylabel('Частота')
plt.grid(axis="y", linestyle="--", alpha=0.5)
plt.show()

In [ ]:
from wordcloud import WordCloud

def plot_wordcloud(sentiment_label):
    text = " ".join(data[data["sentiment"] == sentiment_label]["description"])
    wordcloud = WordCloud(width=500, height=300, background_color="white").generate(text)

    plt.figure(figsize=(7, 4))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"Word Cloud for {sentiment_label} News")
    plt.show()

plot_wordcloud("pos")
plot_wordcloud("neg")
plot_wordcloud("neu")
plot_wordcloud("compound")


### 3.3 Определение главных тем

In [ ]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=50)  # Ограничиваем топ-50 ключевых слов
tfidf_matrix = vectorizer.fit_transform(data["description"])

feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)


In [ ]:
tfidf_df

In [ ]:
top_n = 5
data["top_keywords"] = tfidf_df.apply(lambda row: row.nlargest(top_n).index.tolist(), axis=1)
data